In [1]:
from abc import ABCMeta, abstractmethod
import random
import math
import pandas as pd
import numpy as np

In [2]:
class Datos:

    # Constructor: procesar el fichero para asignar correctamente las
    # variables nominalAtributos, datos y diccionario
    def __init__(self, nombreFichero):

        # Leemos el dataset pasado por argumentos, y lo canvertimos en un DataFrame
        # de pandas
        df = pd.read_csv(nombreFichero)
        # Listas de valores nominales (True), o numericos (False)
        self.nominalAtributos = []
        # conversor de datos nominales a datos numéricos
        self.diccionarios = {}
        # Array numpy de datos convertidos a forma numerica
        self.datos = np.array([])

        # Iterar por los atributos (incluye clase)
        for attr in df.columns:
            # Crear una entrada al diccionario para cada atributo
            self.diccionarios[attr] = {}
            # Se guarda si es atributo nominal o no, en el array nominalAtributos
            if df[attr].dtypes in [np.int64, np.float64]:
                self.nominalAtributos.append(False)
            elif isinstance(df[attr].dtypes, object):
                self.nominalAtributos.append(True)
                value = 1
                # Se iteran los datos únicos obtenidos para cada atributo nominal, de manera ordenanda
                for key in sorted(df[attr].unique()):
                    # Se crea una nueva entrada para cada dato nominal con el valor al cual se va a tranformar
                    self.diccionarios[attr][key] = value
                    value += 1
            else:
                raise ValueError

            # Con el metodo replace, reemplazara los datos de nuestro DataFrame con el mapeo de claves-valor realizadas en diccionarios
            df = df.replace(self.diccionarios)

            # Finalmente se pasa a array de numpy
            self.datos = df.to_numpy()

    def extraeDatos(self, idx):

        # Lista que almacenará los datos correspondientes a los indices pedidos
        listaDatos = []

        for i in idx:
            listaDatos.append(self.datos[i])

        return listaDatos

In [3]:
class Particion():

    # Esta clase mantiene la lista de �ndices de Train y Test para cada partici�n del conjunto de particiones
    def __init__(self):
        self.indicesTrain = []
        self.indicesTest = []

class EstrategiaParticionado:

    # Clase abstracta
    __metaclass__ = ABCMeta

    # Atributos: deben rellenarse adecuadamente para cada estrategia concreta. Se pasan en el constructor
    def __init__(self):
        self.particiones = []

    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada estrategia concreta
    def creaParticiones(self, datos, seed=None):
        pass

class ValidacionSimple(EstrategiaParticionado):

    def __init__(self, numeroEjec, propTest):
        self.numeroEjecuciones = numeroEjec
        self.proporcionTest = propTest
        super().__init__()

    # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado y el n�mero de ejecuciones deseado
    # Devuelve una lista de particiones (clase Particion)
    # TODO: implementar
    def creaParticiones(self, datos, seed=None):
        # Se obtiene el numero de filas y se crea una lista con tantos indices como elementos haya en la lista
        filas = len(datos.datos)
        indices = [i for i in range(0, filas)]
        # Se iteran entre el numero de ejecuciones que se quieren realizar
        for i in range(0, self.numeroEjecuciones):
            # Se hace un mezclado de las filas (indices) del dataset
            random.shuffle(indices)
            # Se crea un nuevo objeto de particionado
            part = Particion()
            # Se añaden los indices de test e indices de entrenamiento en funcion de la proprcion de Test
            part.indicesTrain.extend(indices[0: self.proporcionTest - 1])
            part.indicesTest.extend(indices[self.proporcionTest: filas - 1])
            # Se añade a la lista de particiones, la nueva particion creada
            self.particiones.append(part)

class ValidacionCruzada(EstrategiaParticionado):

    def __init__(self, numParticiones):
        self.numeroParticiones = numParticiones
        super().__init__()

    # Crea particiones segun el metodo de validacion cruzada.
    # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
    # Esta funcion devuelve una lista de particiones (clase Particion)
    # TODO: implementar

    def creaParticiones(self, datos, seed=None):
        # Se obtiene el numero de filas y se calcula cuanto es el tamaño de particion, dependiendo de cuantas queremos
        filas = len(datos.datos)
        indices = [i for i in range(0, filas)]
        tamParticion = math.ceil(filas / self.numeroParticiones)

        # Se itera entre el numero de particiones
        for i in range(0, self.numeroParticiones):

            # Si el test no queda vacio entonces creamos una particion para test
            if len(indices[tamParticion * i: tamParticion * (i + 1)]) != 0:
                # Se crea una nueva particione
                part = Particion()
                # La parte de test la sacamos desde el indice que estamos hasta el indice + 1
                # por el tamaño de la particion
                part.indicesTest.extend(indices[tamParticion *
                                                i: tamParticion * (i + 1)])
                # La parte de train,se divide en la parte anterior y posterior del train
                part.indicesTrain.extend(indices[0: tamParticion * i])
                part.indicesTrain.extend(indices[tamParticion * (i + 1):])

                # Se añade a la lista de particiones
                self.particiones.append(part)


In [5]:
# MOSTRAREMOS SOLO 5 PARTICIONES POR CADA ESTRATEGIA PARA QUE SE PUEDAN APRECIAR
# MEJOR LOS RESULTADOS

datos = Datos("german.csv")
print("Contenido del diccionario:\n" + str(datos.diccionarios))
print("Contenido matriz datos:\n" + str(datos.datos[0]) + "\n" + str(datos.datos[1]))

print("-------------------------------------------------------------------------------------")

estrategiaVS = ValidacionSimple(20, 10)
estrategiaVC = ValidacionCruzada(100)

# Conjunto de indices de train y test para la VS
estrategiaVS.creaParticiones(datos)
print("Indices VS:")
for i in range(0, 5):
    print("Indices " + str(i) + " Train: " +
          str(estrategiaVS.particiones[i].indicesTrain))
    print("Indices " + str(i) + " Test: " +
          str(estrategiaVS.particiones[i].indicesTest))

print("-------------------------------------------------------------------------------------")

# Conjunto de indices de train y test para la VC
estrategiaVC.creaParticiones(datos)
print("Indices VC:")
for i in range(0, 5):
    print("Indices " + str(i) + " Train: " +
          str(estrategiaVC.particiones[i].indicesTrain))
    print("Indices " + str(i) + " Test: " +
          str(estrategiaVC.particiones[i].indicesTest))

Contenido del diccionario:
{'Atr1': {'A11': 1, 'A12': 2, 'A13': 3, 'A14': 4}, 'Atr2': {}, 'Atr3': {'A30': 1, 'A31': 2, 'A32': 3, 'A33': 4, 'A34': 5}, 'Atr4': {'A40': 1, 'A41': 2, 'A410': 3, 'A42': 4, 'A43': 5, 'A44': 6, 'A45': 7, 'A46': 8, 'A48': 9, 'A49': 10}, 'Atr5': {}, 'Atr6': {'A61': 1, 'A62': 2, 'A63': 3, 'A64': 4, 'A65': 5}, 'Atr7': {'A71': 1, 'A72': 2, 'A73': 3, 'A74': 4, 'A75': 5}, 'Atr8': {}, 'Atr9': {'A91': 1, 'A92': 2, 'A93': 3, 'A94': 4}, 'Atr10': {'A101': 1, 'A102': 2, 'A103': 3}, 'Atr11': {}, 'Atr12': {'A121': 1, 'A122': 2, 'A123': 3, 'A124': 4}, 'Atr13': {}, 'Atr14': {'A141': 1, 'A142': 2, 'A143': 3}, 'Atr15': {'A151': 1, 'A152': 2, 'A153': 3}, 'Atr16': {}, 'Atr17': {'A171': 1, 'A172': 2, 'A173': 3, 'A174': 4}, 'Atr18': {}, 'Atr19': {'A191': 1, 'A192': 2}, 'Atr20': {'A201': 1, 'A202': 2}, 'Class': {}}
Contenido matriz datos:
[   1    6    5    5 1169    5    5    4    3    1    4    1   67    3
    2    2    3    1    2    1    1]
[   2   48    3    5 5951    1    3    